In [1]:
import yaml
import numpy as np
import pandas as pd

import cv2
import albumentations as A

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold

from utils import obj
from sdv.tabular.ctgan import CTGAN

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

pd.options.display.max_columns=100

In [92]:
data_path = '../open/'
info = pd.read_excel(data_path+'clinical_info.xlsx')

df_train = pd.read_csv(data_path+'train.csv')
df_test = pd.read_csv(data_path+'test.csv')
sub = pd.read_csv(data_path+'sample_submission.csv')

df_train['img_path'] = df_train['img_path'].apply(lambda x: x.replace('./', '../open/'))
df_test['img_path'] = df_test['img_path'].apply(lambda x: x.replace('./', '../open/'))

In [ ]:
for i, p in enumerate(df_train['img_path']):
    img = cv2.imread(p)
    trans = A.Resize(224, 224)
    img = trans(image=img)['image']
    if size_ls[i][0]>3500:
        

        plt.imshow(img)
        plt.show()

In [41]:
size_ls = []

for i in df_train['img_path']:
    img = cv2.imread(i)
    # trans = A.Compose([
    #                     A.Resize(224, 224), 
                    #    A.HorizontalFlip(p=1),
                    #    A.CoarseDropout(p=1),
                    #    A.Cutout(max_h_size=int(224*0.1), max_w_size=int(224*0.1), num_holes=5, p=0.8),
                    #    ])
    # img = trans(image=img)['image']
    # plt.imshow(img)
    # plt.show()
    size_ls += [img.shape]

In [3]:
info

,항목,코딩,설명
0,ID,NaN,부여번호
1,나이,NaN,진단시 나이
2,수술연월일,연-월-일,NaN
3,진단명,1: ductal\n2: lobular\n3. mucinous \n4. other,histologic type
4,암의 위치,1: Right\n2: Left\n3: Both,NaN
5,암의 개수,1: single\n2: multiple,NaN
6,암의 장경,NaN,"암 Size (mm), 여러 개의 암일 경우 가장 큰 종양의 장경"
7,NG,1: NG1\n2: NG2\n3: NG3,"1은 핵의 모양이 정상에 가까운 경우, 2는 중간, 3은 정상으로부터 매우 달라진경..."
8,HG,1: HG1\n2: HG2\n3: HG3\n4. not graded,"Grade 1 (scores of 3, 4, or 5): 조직학적 분화도가 좋아서 ..."
9,HG_score_1,1: score 1\n2: score 2\n3: score 3\n4: not graded,(Tubule formation)\nScore 1 (>75% of tumor are...


In [7]:
from utils import seed_everything
seed_everything(42)

In [102]:
data_path = '../open/'
df_train = pd.read_csv(data_path+'train.csv')
df_test = pd.read_csv(data_path+'test.csv')

# preprocess outlier
# df_train['PR_Allred_score'] = df_train['PR_Allred_score'].where((0<=df_train['PR_Allred_score']) & (df_train['PR_Allred_score']<=8))

# for col in ['NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3', 'DCIS_or_LCIS_type', 'ER_Allred_score', 'PR_Allred_score', 'HER2_SISH_ratio']:
#     df_train[col].fillna(0, inplace=True)
#     df_test[col].fillna(0, inplace=True)
    # df_train[col].fillna(-1, inplace=True)
    # df_test[col].fillna(-1, inplace=True)

# df_test['암의 장경'].fillna(df_train['암의 장경'].median(), inplace=True)
# df_train['암의 장경'].fillna(df_train['암의 장경'].median(), inplace=True)

# df_train['BRCA_mutation'] = df_train['BRCA_mutation'].fillna(1)
# df_test['BRCA_mutation'] = df_test['BRCA_mutation'].fillna(1)

# for col in ['T_category', 'HER2', 'HER2_IHC', 'HER2_SISH', 'KI-67_LI_percent']:
#     df_train[col].fillna(-1, inplace=True)
#     df_test[col].fillna(-1, inplace=True)

#     df_train[col]+=1
#     df_test[col]+=1

# # remove ER, PR nan value
# df_train = df_train.drop(266).reset_index(drop=True)

df_train['fe1'] = df_train['암의 장경'] / df_train['암의 개수'].fillna(1)
df_test['fe1'] = df_test['암의 장경'] / df_test['암의 개수'].fillna(1)



for df in [df_train, df_test]:
    # df['나이'] = pd.cut(df['나이'], [20, 30, 40, 50, 60, 70, 80, 100])
    df['암의 장경'] = pd.cut(df['암의 장경'], [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200])
    df['fe1'] = pd.cut(df['fe1'], [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200])

FOLDS = 5

def basic_set():
    use_features = [
                    '나이', 
                    '진단명', '암의 위치', '암의 개수', '암의 장경', 'NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3',
                    'DCIS_or_LCIS_여부', 'DCIS_or_LCIS_type', 'T_category', 
                    'ER', 'ER_Allred_score', 'PR', 'PR_Allred_score', 
                    'KI-67_LI_percent', 
                    'HER2', 'HER2_IHC', 'HER2_SISH', 'HER2_SISH_ratio', 'BRCA_mutation', 
                    'fe1', 
                    'fe2',
                    # 'img_height', 'img_width',
                    # 'BRCA_mutation2',
                    ]

    cat_features = ['암의 장경', 'fe1']
    return use_features, cat_features

df_train['due_date'] = 2022 - pd.to_datetime(df_train['수술연월일']).dt.year
df_test['due_date'] = 2022 - pd.to_datetime(df_test['수술연월일']).dt.year
# df_train['due_date2'] = (2022-pd.to_datetime(df_train['수술연월일']).dt.year)*12 + pd.to_datetime(df_train['수술연월일']).dt.month
# df_test['due_date2'] = (2022-pd.to_datetime(df_test['수술연월일']).dt.year)*12 + pd.to_datetime(df_test['수술연월일']).dt.month

df_train['date_year'] = pd.to_datetime(df_train['수술연월일']).dt.year
df_test['date_year'] = pd.to_datetime(df_test['수술연월일']).dt.year

# df_train['fe2'] = df_train['DCIS_or_LCIS_여부'].astype(str) + '_' + df_train['DCIS_or_LCIS_type'].astype(str)
# df_test['fe2'] = df_test['DCIS_or_LCIS_여부'].astype(str) + '_' + df_test['DCIS_or_LCIS_type'].astype(str)

# df_train['fe2'] = 100 - df_train['KI-67_LI_percent']
# df_test['fe2'] = 100 - df_test['KI-67_LI_percent']
# df_train['fe2'] = df_train['KI-67_LI_percent'] / 100
# df_test['fe2'] = df_test['KI-67_LI_percent'] / 100

# df_train[['HG_score_1', 'HG_score_2', 'HG_score_3']]

# df_train['fe2'] = (df_train['HG_score_1'] > df_train['HG_score_3']).astype(int)
# df_test['fe2'] = (df_test['HG_score_1'] > df_test['HG_score_3']).astype(int)

# df_train['fe2'] = ((df_train['ER']==1) & (df_train['PR']==1)).astype(int)
# df_test['fe2'] = ((df_test['ER']==1) & (df_test['PR']==1)).astype(int)

# df_train['수술당시나이'] = df_train['나이'] - df_train['due_date']
# df_test['수술당시나이'] = df_test['나이'] - df_test['due_date']

# df_train['암의 장경/개수'] = df_train['암의 장경'] / df_train['암의 개수']
# df_test['암의 장경/개수'] = df_test['암의 장경'] / df_test['암의 개수']

for col in [
                    '나이', '진단명', '암의 위치', '암의 개수', 'NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3',
                    'DCIS_or_LCIS_여부', 'DCIS_or_LCIS_type', 'T_category', 
                    'ER', 'ER_Allred_score', 'PR', 'PR_Allred_score', 
                    'HER2', 'HER2_IHC', 'HER2_SISH', 'HER2_SISH_ratio', 'BRCA_mutation', 'due_date', 'date_year',
                    'fe2',
                    ]:
    df_test[col] = df_test[col].map(df_train[col].value_counts(True))
    df_train[col] = df_train[col].map(df_train[col].value_counts(True))

skf = StratifiedKFold(n_splits=FOLDS, random_state=42, shuffle=True)
splits = list(skf.split(df_train, df_train['N_category']))
# labels = df_train['N_category'].astype(str) + '_' + df_train['due_date'].astype(str)
# splits = list(skf.split(df_train, labels))


fi_df = pd.DataFrame()
oof = np.zeros(len(df_train))
preds = np.zeros(len(df_test))

for fold in range(FOLDS):
    print('start fold :', fold)
    test = df_test.copy()
    
    use_features, cat_features = basic_set()
    use_features += ['due_date', 'date_year']
    # use_features += ['due_date', 'date_year', '암의 장경/개수']

    tt = df_train.loc[splits[fold][0], use_features].reset_index(drop=True)
    tt_target = df_train.loc[splits[fold][0], 'N_category'].values
    vv = df_train.loc[splits[fold][1], use_features].reset_index(drop=True)
    vv_target = df_train.loc[splits[fold][1], 'N_category'].values

    for col in ['암의 장경', 'fe1']:
        tmp_dict = {j:i for i,j in enumerate(np.unique(df_train[col].astype(str)))}
        tt[col] = tt[col].astype(str).map(tmp_dict).astype(int)
        vv[col] = vv[col].astype(str).map(tmp_dict).astype(int)
        test[col] = test[col].astype(str).map(tmp_dict).astype(int)

    # generate dataset
    # tt = tt.fillna(0)
    # vv = vv.fillna(0)
    # test = test.fillna(0)

    # df = pd.concat([tt, pd.DataFrame(tt_target, columns=['target'])], axis=1)

    # # discrete_columns = ['암의 장경', 'target']

    # # ctgan = CTGAN(epochs=10)
    # ctgan = CopulaGAN(epochs=10)
    # ctgan.fit(df)
    # # Create synthetic data
    # synthetic_data = ctgan.sample(50, randomize_samples=False)
    # df = pd.concat([df, synthetic_data], ignore_index=True)
    # tt_target = df['target'].values
    # tt = df.drop(columns='target')
    
    # ctgan = pd.read_csv(f'../open/ctgan_csv/50/400/{fold}.csv')
    # tt = pd.concat([tt, ctgan[use_features]], ignore_index=False)
    # tt_target = np.concatenate([tt_target, ctgan['target'].values])
    
    # tt['암의 장경2'] = tt['암의 장경'].isnull()
    # vv['암의 장경2'] = vv['암의 장경'].isnull()
    # test['암의 장경2'] = test['암의 장경'].isnull()

    # for df in [tt, vv, test]:
    #     # df['나이_due_date'] = df['나이'].astype(str) + '_' + df['due_date'].astype(str)
    #     df['암'] = df['암의 위치'].astype(str) + '_' + df['암의 장경'].astype(str)
    # df = pd.concat([tt, vv])
    # tmp_dict = {v:i for i, v in enumerate(np.unique(df['암']))}
    # for col in ['암']:
    #     tt[col] = tt[col].astype(str).map(tmp_dict)
    #     vv[col] = vv[col].astype(str).map(tmp_dict)
    #     test[col] = test[col].astype(str).map(tmp_dict)
    
    # use_features += ['암']
    
    # tmp_dict = df_train.loc[splits[fold][0]].groupby('due_date')['N_category'].mean()
    # tt['진단명_target'] = tt['due_date'].map(tmp_dict)
    # vv['진단명_target'] = vv['due_date'].map(tmp_dict)
    # tmp_dict = df_train.groupby('due_date')['N_category'].mean()
    # test['진단명_target'] = test['due_date'].map(tmp_dict)
    # use_features += ['진단명_target']
    


    test = test[use_features].reset_index(drop=True)

    # lgb = LGBMClassifier(
    #                     n_estimators=5000,
    #                     learning_rate=0.03,
    #                     max_depth=-1,
    #                     num_leaves=64,
    #                     )

    # lgb.fit(tt, tt_target, eval_set=[(tt, tt_target), (vv, vv_target)], verbose=1000, early_stopping_rounds=100)
    # oof[splits[fold][1]] += lgb.predict_proba(vv)[:, 1]
    # preds += lgb.predict_proba(test)[:, 1] / 5
    # fi_df = pd.concat([fi_df, pd.DataFrame(zip(use_features, lgb.feature_importances_), columns=['feature', 'value'])])
    # tt[cat_features] = tt[cat_features].fillna(-99).astype(int)
    # vv[cat_features] = vv[cat_features].fillna(-99).astype(int)
    cat = CatBoostClassifier(
                            n_estimators=5000,
                            learning_rate=0.03,
                            # border_count=4,
                            # depth=4,
                            # task_type="GPU",
                            # devices='0',
                            # one_hot_max_size=2,
                            # objective='logloss',
                            # eval_metric='F1',
                            # reg_lambda=0.01,
                            # random_seed=1028,
                            allow_writing_files=False,
                            logging_level='Silent',
                            )
    cat.fit(tt, tt_target, eval_set=[(vv, vv_target)], verbose=1000, early_stopping_rounds=500,
            # cat_features=cat_features, 
            )
    # cat.fit(tt[lambda x: x['due_date']<14], tt_target[tt['due_date']<14], eval_set=[(tt, tt_target), (vv, vv_target)], verbose=1000, early_stopping_rounds=500,
    #         # cat_features=cat_features
    #         )

            
    oof[splits[fold][1]] += cat.predict_proba(vv)[:, 1]
    preds += cat.predict_proba(test)[:, 1] / FOLDS
    fi_df = pd.concat([fi_df, pd.DataFrame(zip(use_features, cat.feature_importances_), columns=['feature', 'value'])])

start fold : 0
start fold : 1
start fold : 2
start fold : 3
start fold : 4


In [103]:
0.8449961249031226

0.8449961249031226

In [104]:
f1_score(df_train['N_category'],
        np.where(oof>0.5, 1, 0),
        average='macro'
        )

0.8349985149866348

In [105]:
f1_score(df_train['N_category'],
        np.where(oof>0.5, 1, 0),
        average='macro'
        )

0.8349985149866348

In [10]:
b = pd.read_csv('../submit/catboost_ctgan60e_600s.csv')
c = pd.Series(preds.round())


np.not_equal(c, b['N_category']).sum()

8

In [11]:
f1_score(df_train['N_category'],
        np.where(oof>0.5, 1, 0),
        average='macro'
        )

0.852

In [43]:
fi_df.groupby('feature').mean().reset_index().sort_values('value', ascending=False)

,feature,value
19,due_date,15.119064
18,date_year,10.838392
14,NG,9.709931
11,HG_score_2,8.582237
2,DCIS_or_LCIS_여부,6.257614
13,KI-67_LI_percent,5.265695
21,나이,4.345035
24,암의 장경,3.893144
22,암의 개수,3.721905
10,HG_score_1,3.676370


In [13]:
pd.Series(np.where(oof>0.5, 1, 0)).value_counts(True)

0    0.514
1    0.486
dtype: float64

In [14]:
df_train['N_category'].value_counts(True)

1    0.514
0    0.486
Name: N_category, dtype: float64

In [15]:
pd.Series(np.where(preds>0.4, 1, 0)).value_counts(True)

1    0.524
0    0.476
dtype: float64

In [16]:
pd.Series(np.where(preds>0.5, 1, 0)).value_counts(True)

0    0.564
1    0.436
dtype: float64

In [17]:
df_train

,ID,img_path,mask_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,HG_score_3,DCIS_or_LCIS_여부,DCIS_or_LCIS_type,T_category,ER,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category,fe1,due_date,date_year
0,BC_01_0001,./train_imgs/BC_01_0001.png,-,0.024,2015-10-23,0.857,0.495,0.867,19.0,0.516333,0.310722,0.226872,0.549559,0.701427,0.318,NaN,0.602410,0.81982,0.388336,0.641642,0.127971,12.0,0.752535,0.321721,NaN,NaN,NaN,0,19.0,0.085,0.085
1,BC_01_0002,./train_imgs/BC_01_0002.png,-,0.031,2015-10-28,0.857,0.472,0.867,22.0,0.277134,0.207877,0.680617,0.297357,0.122942,0.599,NaN,0.306225,0.18018,NaN,0.358358,NaN,70.0,0.752535,0.257172,NaN,NaN,NaN,1,22.0,0.085,0.085
2,BC_01_0003,./train_imgs/BC_01_0003.png,-,0.009,2015-10-29,0.857,0.495,0.867,NaN,0.516333,NaN,NaN,NaN,NaN,0.083,0.452381,0.071285,0.81982,0.419630,0.641642,0.175503,7.0,0.752535,0.321721,NaN,NaN,0.927273,0,NaN,0.085,0.085
3,BC_01_0004,./train_imgs/BC_01_0004.png,-,0.036,2016-03-08,0.857,0.495,0.867,0.0,0.277134,0.207877,0.680617,0.297357,0.172338,0.083,0.452381,0.071285,0.18018,NaN,0.358358,NaN,1.0,0.247465,0.089139,NaN,NaN,NaN,0,0.0,0.087,0.087
4,BC_01_0005,./train_imgs/BC_01_0005.png,-,0.027,2015-10-30,0.857,0.495,0.867,8.0,0.516333,0.481400,0.680617,0.549559,0.701427,0.318,NaN,0.602410,0.81982,0.388336,0.358358,NaN,8.0,0.247465,0.331967,0.129555,0.005714,NaN,0,8.0,0.085,0.085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,BC_01_3464,./train_imgs/BC_01_3464.png,-,0.021,2006-12-22,0.857,0.495,0.867,25.0,0.206533,0.310722,0.226872,0.153084,0.701427,0.599,NaN,0.306225,0.81982,NaN,0.358358,NaN,NaN,0.752535,0.257172,NaN,NaN,NaN,1,25.0,0.004,0.004
996,BC_01_3482,./train_imgs/BC_01_3482.png,-,0.047,2006-11-17,0.857,0.472,0.867,7.0,0.206533,0.310722,0.092511,0.153084,0.701427,0.599,NaN,0.602410,0.81982,NaN,0.641642,NaN,NaN,0.752535,0.257172,NaN,NaN,NaN,0,7.0,0.004,0.004
997,BC_01_3485,./train_imgs/BC_01_3485.png,-,0.021,2006-11-10,0.857,0.495,0.867,15.0,0.206533,0.310722,0.226872,0.153084,0.172338,0.599,NaN,0.602410,0.81982,NaN,0.641642,NaN,NaN,0.752535,0.257172,NaN,NaN,NaN,1,15.0,0.004,0.004
998,BC_01_3502,./train_imgs/BC_01_3502.png,-,0.040,2006-09-22,0.857,0.472,0.867,7.0,0.206533,0.310722,0.092511,0.549559,0.701427,0.599,NaN,0.602410,0.18018,NaN,0.358358,NaN,NaN,0.752535,0.257172,NaN,NaN,NaN,0,7.0,0.004,0.004


In [23]:
sub['N_category'] = np.where(preds>=0.5, 1, 0)

In [25]:
sub.to_csv('../submit/catboost_l1_01__ctgan50e_400s__round45.csv', index=False)

In [21]:
np.save('../submit/cat_oof.npy', oof)

In [22]:
np.save('../submit/cat_preds.npy', preds)

In [4]:
df_train = pd.read_csv(data_path+'df_train.csv')
df_test = pd.read_csv(data_path+'df_test.csv')

In [6]:
df_train['due_date'] = 2022 - pd.to_datetime(df_train['수술연월일']).dt.year
df_test['due_date'] = 2022 - pd.to_datetime(df_test['수술연월일']).dt.year

In [25]:
df_train['암의 위치']df_train['암의 개수'].astype(str)

0      2_1
1      1_1
2      2_1
3      2_1
4      2_1
      ... 
995    2_1
996    1_1
997    2_1
998    1_1
999    2_1
Length: 1000, dtype: object

In [13]:
seed_oof = []
seed_pred = []
for seed in [1028, 42, 204, 1510, 99]:

    df_train = pd.read_csv(data_path+'train.csv')
    df_test = pd.read_csv(data_path+'test.csv')

    # preprocess outlier
    # df_train['PR_Allred_score'] = df_train['PR_Allred_score'].where((0<=df_train['PR_Allred_score']) & (df_train['PR_Allred_score']<=8))

    # for col in ['NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3', 'DCIS_or_LCIS_type', 'ER_Allred_score', 'PR_Allred_score', 'HER2_SISH_ratio']:
    #     df_train[col].fillna(0, inplace=True)
    #     df_test[col].fillna(0, inplace=True)
        # df_train[col].fillna(-1, inplace=True)
        # df_test[col].fillna(-1, inplace=True)

    # df_test['암의 장경'].fillna(df_train['암의 장경'].median(), inplace=True)
    # df_train['암의 장경'].fillna(df_train['암의 장경'].median(), inplace=True)

    # df_train['BRCA_mutation'] = df_train['BRCA_mutation'].fillna(1)
    # df_test['BRCA_mutation'] = df_test['BRCA_mutation'].fillna(1)

    # for col in ['T_category', 'HER2', 'HER2_IHC', 'HER2_SISH', 'KI-67_LI_percent']:
    #     df_train[col].fillna(-1, inplace=True)
    #     df_test[col].fillna(-1, inplace=True)

    #     df_train[col]+=1
    #     df_test[col]+=1

    # # remove ER, PR nan value
    # df_train = df_train.drop(266).reset_index(drop=True)

    FOLDS = 5

    def basic_set():
        use_features = [
                        '나이', 
                        '진단명', '암의 위치', '암의 개수', '암의 장경', 'NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3',
                        'DCIS_or_LCIS_여부', 'DCIS_or_LCIS_type', 'T_category', 
                        'ER', 'ER_Allred_score', 'PR', 'PR_Allred_score', 
                        'KI-67_LI_percent', 
                        'HER2', 'HER2_IHC', 'HER2_SISH', 'HER2_SISH_ratio', 'BRCA_mutation', 
                        # 'img_height', 'img_width',
                        # 'BRCA_mutation2',
                        ]

        cat_features = ['나이', '진단명', '암의 위치', '암의 개수', '암의 장경', 'NG', 'HG', 'HG_score_1',
                                'HG_score_2', 'HG_score_3', 'DCIS_or_LCIS_여부', 'DCIS_or_LCIS_type',
                                'T_category', 'ER', 'ER_Allred_score', 'PR', 'PR_Allred_score',
                                'HER2', 'HER2_IHC', 'HER2_SISH','BRCA_mutation']
        cat_features = ['진단명']
        return use_features, cat_features

    df_train['due_date'] = 2022 - pd.to_datetime(df_train['수술연월일']).dt.year
    df_test['due_date'] = 2022 - pd.to_datetime(df_test['수술연월일']).dt.year
    # df_train['due_date2'] = (2022-pd.to_datetime(df_train['수술연월일']).dt.year)*12 + pd.to_datetime(df_train['수술연월일']).dt.month
    # df_test['due_date2'] = (2022-pd.to_datetime(df_test['수술연월일']).dt.year)*12 + pd.to_datetime(df_test['수술연월일']).dt.month

    df_train['date_year'] = pd.to_datetime(df_train['수술연월일']).dt.year
    df_test['date_year'] = pd.to_datetime(df_test['수술연월일']).dt.year

    # df_train['수술당시나이'] = df_train['나이'] - df_train['due_date']
    # df_test['수술당시나이'] = df_test['나이'] - df_test['due_date']

    # df_train['암의 장경/개수'] = df_train['암의 장경'] / df_train['암의 개수']
    # df_test['암의 장경/개수'] = df_test['암의 장경'] / df_test['암의 개수']

    for col in [
                        '나이', '진단명', '암의 위치', '암의 개수', 'NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3',
                        'DCIS_or_LCIS_여부', 'DCIS_or_LCIS_type', 'T_category', 
                        'ER', 'ER_Allred_score', 'PR', 'PR_Allred_score', 
                        'HER2', 'HER2_IHC', 'HER2_SISH', 'HER2_SISH_ratio', 'BRCA_mutation', 'due_date', 'date_year']:
        df_test[col] = df_test[col].map(df_train[col].value_counts(True))
        df_train[col] = df_train[col].map(df_train[col].value_counts(True))

    skf = StratifiedKFold(n_splits=FOLDS, random_state=seed, shuffle=True)
    splits = list(skf.split(df_train, df_train['N_category']))
    # labels = df_train['N_category'].astype(str) + '_' + df_train['due_date'].astype(str)
    # splits = list(skf.split(df_train, labels))


    fi_df = pd.DataFrame()
    oof = np.zeros(len(df_train))
    preds = np.zeros(len(df_test))

    for fold in range(FOLDS):
        print('start fold :', fold)
        test = df_test.copy()
        
        use_features, cat_features = basic_set()
        use_features += ['due_date', 'date_year']
        # use_features += ['due_date', 'date_year', '암의 장경/개수']

        tt = df_train.loc[splits[fold][0], use_features].reset_index(drop=True)
        tt_target = df_train.loc[splits[fold][0], 'N_category'].values
        vv = df_train.loc[splits[fold][1], use_features].reset_index(drop=True)
        vv_target = df_train.loc[splits[fold][1], 'N_category'].values

        for df in [tt, vv, test]:
            # df['나이'] = pd.cut(df['나이'], [20, 30, 40, 50, 60, 70, 80, 100])
            df['암의 장경'] = pd.cut(df['암의 장경'], [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200])

            # df['due_date'] = pd.cut(df['due_date'], [0, 5, 10, 15])
            # df['date_year'] = pd.cut(df['date_year'], [0, 2005, 2010, 2015, 2020, 2030])
            # df['암의 장경'] = pd.cut(df['암의 장경'], [0, 20, 40, 60, 80, 100, 200])
            # df['암의 장경/개수'] = pd.cut(df['암의 장경/개수'], [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200])
            # df['KI-67_LI_percent'] = pd.cut(df['KI-67_LI_percent'], [0, 20, 40, 60, 80, 100])
            
        # for col in ['나이', '암의 장경', '암의 장경/개수']:
        # for col in ['나이', '암의 장경', 'KI-67_LI_percent']:
        # for col in ['나이', '암의 장경']:
        for col in ['암의 장경']:
            tmp_dict = {j:i for i,j in enumerate(np.unique(tt[col].astype(str)))}
            tt[col] = tt[col].astype(str).map(tmp_dict)
            vv[col] = vv[col].astype(str).map(tmp_dict)
            test[col] = test[col].astype(str).map(tmp_dict)

        # generate dataset
        # tt = tt.fillna(0)
        # vv = vv.fillna(0)
        # test = test.fillna(0)

        # df = pd.concat([tt, pd.DataFrame(tt_target, columns=['target'])], axis=1)

        # # discrete_columns = ['암의 장경', 'target']

        # # ctgan = CTGAN(epochs=10)
        # ctgan = CopulaGAN(epochs=10)
        # ctgan.fit(df)
        # # Create synthetic data
        # synthetic_data = ctgan.sample(50, randomize_samples=False)
        # df = pd.concat([df, synthetic_data], ignore_index=True)
        # tt_target = df['target'].values
        # tt = df.drop(columns='target')
        ctgan = pd.read_csv(f'../open/ctgan_csv/100/500/{fold}.csv')
        tt = pd.concat([tt, ctgan[use_features]], ignore_index=False)
        tt_target = np.concatenate([tt_target, ctgan['target'].values])
        
        # tt['암의 장경2'] = tt['암의 장경'].isnull()
        # vv['암의 장경2'] = vv['암의 장경'].isnull()
        # test['암의 장경2'] = test['암의 장경'].isnull()

        # for df in [tt, vv, test]:
        #     # df['나이_due_date'] = df['나이'].astype(str) + '_' + df['due_date'].astype(str)
        #     df['암'] = df['암의 위치'].astype(str) + '_' + df['암의 장경'].astype(str)
        # df = pd.concat([tt, vv])
        # tmp_dict = {v:i for i, v in enumerate(np.unique(df['암']))}
        # for col in ['암']:
        #     tt[col] = tt[col].astype(str).map(tmp_dict)
        #     vv[col] = vv[col].astype(str).map(tmp_dict)
        #     test[col] = test[col].astype(str).map(tmp_dict)
        
        # use_features += ['암']
        
        # tmp_dict = df_train.loc[splits[fold][0]].groupby('due_date')['N_category'].mean()
        # tt['진단명_target'] = tt['due_date'].map(tmp_dict)
        # vv['진단명_target'] = vv['due_date'].map(tmp_dict)
        # tmp_dict = df_train.groupby('due_date')['N_category'].mean()
        # test['진단명_target'] = test['due_date'].map(tmp_dict)
        # use_features += ['진단명_target']
        


        test = test[use_features].reset_index(drop=True)

        # lgb = LGBMClassifier(
        #                     n_estimators=5000,
        #                     learning_rate=0.03,
        #                     max_depth=-1,
        #                     num_leaves=64,
        #                     )

        # lgb.fit(tt, tt_target, eval_set=[(tt, tt_target), (vv, vv_target)], verbose=1000, early_stopping_rounds=100)
        # oof[splits[fold][1]] += lgb.predict_proba(vv)[:, 1]
        # preds += lgb.predict_proba(test)[:, 1] / 5
        # fi_df = pd.concat([fi_df, pd.DataFrame(zip(use_features, lgb.feature_importances_), columns=['feature', 'value'])])
        # tt[cat_features] = tt[cat_features].fillna(-99).astype(int)
        # vv[cat_features] = vv[cat_features].fillna(-99).astype(int)
        cat = CatBoostClassifier(
                                n_estimators=5000,
                                learning_rate=0.03,
                                # objective='logloss',
                                # eval_metric='F1',
                                # reg_lambda=0.01,
                                # random_seed=1028,
                                allow_writing_files=False,
                                logging_level='Silent',
                                )
        cat.fit(tt, tt_target, eval_set=[(tt, tt_target), (vv, vv_target)], verbose=1000, early_stopping_rounds=500,
                # cat_features=cat_features
                )
        # cat.fit(tt[lambda x: x['due_date']<14], tt_target[tt['due_date']<14], eval_set=[(tt, tt_target), (vv, vv_target)], verbose=1000, early_stopping_rounds=500,
        #         # cat_features=cat_features
        #         )

                
        oof[splits[fold][1]] += cat.predict_proba(vv)[:, 1]
        preds += cat.predict_proba(test)[:, 1] / FOLDS
        fi_df = pd.concat([fi_df, pd.DataFrame(zip(use_features, cat.feature_importances_), columns=['feature', 'value'])])
        
        
    seed_oof.append(oof)
    seed_pred.append(preds)

start fold : 0
start fold : 1
start fold : 2
start fold : 3
start fold : 4
start fold : 0
start fold : 1
start fold : 2
start fold : 3
start fold : 4
start fold : 0
start fold : 1
start fold : 2
start fold : 3
start fold : 4
start fold : 0
start fold : 1
start fold : 2
start fold : 3
start fold : 4
start fold : 0
start fold : 1
start fold : 2
start fold : 3
start fold : 4


In [31]:
f1_score(df_train['N_category'],
         np.where(np.mean(seed_oof, 0)>0.5, 1, 0),
         average='macro')

0.8459778208061961

In [17]:
b = pd.read_csv('../submit/catboost_ctgan60e_600s.csv')
c = pd.Series(np.where(np.mean(seed_pred, 0)>=0.5, 1, 0))


np.not_equal(c, b['N_category']).sum()

11

In [18]:
sub['N_category'] = np.where(np.mean(seed_pred, 0)>=0.5, 1, 0)

In [19]:
sub.to_csv('../submit/catboost_ctgan100e_500s__5seed__F1.csv', index=False)